# Your first RAG application

In [2]:
import pandas as pd
import numpy as np 

In [3]:
!ollama list 

NAME              ID              SIZE      MODIFIED     
mistral:latest    f974a74358d6    4.1 GB    24 hours ago    


In [5]:
query = "How old is Julen Haggs"
doc   = "The place of birth of Julen Haggs is Reykjavyk"

template = f"""You are an expert in geography. Your task is to reply to customer questions. Your source of truth are only the documents provided, only generate answers that come from the documents. 
    
     user query: {query} 
     documents: Use only the following document to answer the question: DOCUMENT START: {doc} END DOCUMENT
     """

In [6]:
import requests

res = requests.post("http://localhost:11434/api/generate", json={
    "model": "mistral",
    "prompt": template,
    "stream": False
})

print(res.json()["response"])

 I'm sorry for any inconvenience, but the provided document does not contain information about Julen Haggs' age. It only specifies his place of birth as Reykjavik.
